In [ ]:
import subprocess
import os

import netCDF4
import numpy as np
import glob
import time
import matplotlib.pyplot as plt
import copy
import xarray as xr
from datetime import datetime, timedelta 
#from ocean_c_lab_tools import *
from celluloid import Camera 
import PyCO2SYS as csys
import seawater as sw
import xesmf as xe

In [ ]:
r2=xr.open_mfdataset('/global/cfs/cdirs/m4632/uheede/Vedur_data/ig-is_202406-8/netcdf_files/ig-is_202408*_rhum.nc')
ssr=xr.open_mfdataset('/global/cfs/cdirs/m4632/uheede/Vedur_data/ig-is_202406-8/netcdf_files/ig-is_202408*_swavr.nc')
str=xr.open_mfdataset('/global/cfs/cdirs/m4632/uheede/Vedur_data/ig-is_202406-8/netcdf_files/ig-is_202408*_lwavr.nc')
t2m=xr.open_mfdataset('/global/cfs/cdirs/m4632/uheede/Vedur_data/ig-is_202406-8/netcdf_files/ig-is_202408*_t2m.nc')
tp=xr.open_mfdataset('/global/cfs/cdirs/m4632/uheede/Vedur_data/ig-is_202406-8/netcdf_files/ig-is_202408*_tp.nc')
u10=xr.open_mfdataset('/global/cfs/cdirs/m4632/uheede/Vedur_data/ig-is_202406-8/netcdf_files/ig-is_202408*_u10.nc')
v10=xr.open_mfdataset('/global/cfs/cdirs/m4632/uheede/Vedur_data/ig-is_202406-8/netcdf_files/ig-is_202408*_v10.nc')




In [ ]:
ssr.load()
str.load()

In [ ]:
valid_time_seconds = (r2['valid_time'].astype('datetime64[s]') - np.datetime64('1970-01-01T00:00:00Z')).astype(int)/1e9


In [ ]:
r2['time']=valid_time_seconds
ssr['time']=valid_time_seconds
str['time']=valid_time_seconds
t2m['time']=valid_time_seconds
tp['time']=valid_time_seconds
u10['time']=valid_time_seconds
v10['time']=valid_time_seconds

In [ ]:
ssr['time']=valid_time_seconds
ssr['time']

In [ ]:
lon_min=np.min(r2.longitude.load())
lon_max=np.max(r2.longitude.load())

lon_step=((lon_max-lon_min)/len(r2.x))

lat_min=np.min(r2.latitude.load())
lat_max=np.max(r2.latitude.load())
lat_step=((lat_max-lat_min)/len(r2.y))

In [ ]:
ds_out = xr.Dataset(
    {
        "lon": (["lon"], np.arange(lon_min, lon_max, lon_step), {"units": "degrees_north"}),
        "lat": (["lat"], np.arange(lat_min, lat_max, lat_step), {"units": "degrees_east"}),
    }
)

In [ ]:
ds_out

In [ ]:
regridder = xe.Regridder(r2, ds_out, "bilinear")
r2_regrid=regridder(r2['rhum'])

ssr_regrid=regridder(ssr['swavr'])
str_regrid=regridder(str['lwavr'])
t2m_regrid=regridder(t2m['t2m'])
tp_regrid=regridder(tp['tp']*0.001) # convert from kg/m2 to m
u10_regrid=regridder(u10['u10'])
v10_regrid=regridder(v10['v10'])

In [ ]:
r2_regrid

In [ ]:
cf=plt.contourf(t2m_regrid.isel(valid_time=0))
plt.colorbar(cf)

In [ ]:
p=101000
e=610.94*np.exp(17.625*(t2m_regrid-273.15)/((t2m_regrid-273.15)+243.04))*r2_regrid
q = (0.622 * e) / (p - (1 - 0.622) * e)


In [ ]:
cf=plt.contourf(q.isel(valid_time=0))
plt.colorbar(cf)

In [ ]:
sh2_regrid = q

In [ ]:
r2_regrid.attrs['long name'] = '2 metre relative humidity'
r2_regrid=r2_regrid.reset_coords(names="heightAboveGround",drop=True)
#r2_regrid=r2_regrid.rename({"rhum":'r2'})

sh2_regrid.attrs['long name'] = '2 metre specific humidity'
sh2_regrid=sh2_regrid.reset_coords(names="heightAboveGround",drop=True)

ssr_regrid.attrs['long name'] = 'Surface net short-wave (solar) radiation'
ssr_regrid=ssr_regrid.reset_coords(names="heightAboveGround",drop=True)
#ssr_regrid=ssr_regrid.rename({"swavr":'ssr'})

str_regrid.attrs['long name'] = 'Surface long-wave (thermal) radiation downwards'
str_regrid=str_regrid.reset_coords(names="heightAboveGround",drop=True)
#str_regrid=str_regrid.rename({"lwavr":'str'})

u10_regrid.attrs['long name'] = '10 metre U wind component'
u10_regrid=u10_regrid.reset_coords(names="heightAboveGround",drop=True)

v10_regrid.attrs['long name'] = '10 metre V wind component'
v10_regrid=v10_regrid.reset_coords(names="heightAboveGround",drop=True)

t2m_regrid.attrs['long name'] = '2 metre temperature'
t2m_regrid=t2m_regrid.reset_coords(names="heightAboveGround",drop=True)

tp_regrid.attrs['long name'] = 'total precipitation'
tp_regrid=tp_regrid.reset_coords(names="heightAboveGround",drop=True)


In [ ]:
print(u10_regrid.time)
print(v10_regrid.time)
print(t2m_regrid.time)
print(tp_regrid.time)
print(ssr_regrid.time)
print(str_regrid.time)
print(sh2_regrid.time)

In [ ]:
vedur_2024 = xr.Dataset({
    'u10': u10_regrid.load().squeeze(),
    'v10': v10_regrid.load().squeeze(),
    't2m': t2m_regrid.load().squeeze(),
    'strd': str_regrid.load().squeeze(),
    'ssr': ssr_regrid.load().squeeze(),
    'sh2': sh2_regrid.load().squeeze(),
    'tp': tp_regrid.load().squeeze(),
    'longitude': u10_regrid.load().lon,
    'latitude': u10_regrid.load().lat})

In [ ]:
sh2_regrid

In [ ]:
#vedur_2024 = vedur_2024.set_index(time='valid_time')

In [ ]:
vedur_2024.to_netcdf('/global/cfs/cdirs/m4632/uheede/Vedur_data/ig-is_202406-8/netcdf_files/vedur_202408.nc')




In [ ]:
vedur_2024

In [ ]:
720/24